In [1]:
ROWID = ['f_0']
DATE = ['f_1']
CATEGORIES = [ f'f_{i}' for i in range(2,33) ]
BINARY = [ f'f_{i}' for i in range(33,42) ]
NUMERICAL = [ f'f_{i}' for i in range(42,80) ]
IS_CLICKED = ['is_clicked']
IS_INSTALLED =['is_installed']

In [2]:
import pandas as pd

In [3]:
missing = pd.read_csv('../Data/miss_combine.csv')

In [4]:
# for i,k in enumerate(missing.isnull().sum()):
#     print(f"f_{i} {k}",end='#\n')

In [5]:
data_click = missing
data_install = missing# missing[missing[DATE[0]]>55]

In [46]:
len(data_click)

3485852

In [47]:
len(data_install)

3485852

In [48]:
data_click['f_30'].fillna(data_click['f_30'].mode()[0],inplace=True)
data_click['f_31'].fillna(data_click['f_31'].mode()[0],inplace=True)

### CAT

In [6]:
missing['f_30'].fillna(missing['f_30'].mode()[0],inplace=True)
missing['f_31'].fillna(missing['f_31'].mode()[0],inplace=True)


### NUM

In [7]:
from tqdm import tqdm
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in tqdm(fmiss,desc="NUM IMPUTE"):
    missing[f].fillna(missing[f].mean(),inplace=True)

NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 43.88it/s]


In [8]:
missing.columns

Index(['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9',
       'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 'f_17', 'f_18',
       'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_27',
       'f_28', 'f_29', 'f_30', 'f_31', 'f_32', 'f_33', 'f_34', 'f_35', 'f_36',
       'f_37', 'f_38', 'f_39', 'f_40', 'f_41', 'f_42', 'f_43', 'f_44', 'f_45',
       'f_46', 'f_47', 'f_48', 'f_49', 'f_50', 'f_51', 'f_52', 'f_53', 'f_54',
       'f_55', 'f_56', 'f_57', 'f_58', 'f_59', 'f_60', 'f_61', 'f_62', 'f_63',
       'f_64', 'f_65', 'f_66', 'f_67', 'f_68', 'f_69', 'f_70', 'f_71', 'f_72',
       'f_73', 'f_74', 'f_75', 'f_76', 'f_77', 'f_78', 'f_79', 'is_clicked',
       'is_installed'],
      dtype='object')

In [9]:
max(data_install.isnull().sum())

0

In [10]:
data = missing

In [ ]:
data.groupby('f_1').sum()[IS_CLICKED+IS_INSTALLED]

### This is around 10% of the Data

In [7]:
# click_validation = data_click[(data_click[DATE[0]]==66)]

In [11]:
click_train = data_click

In [12]:
# install_validation = data_install[(data_click[DATE[0]]==66)]
install_train = data_install#[data_install[DATE[0]]>55]

In [13]:
len(install_train)

3485852

In [14]:
len(install_validation)

NameError: name 'install_validation' is not defined

In [14]:
len(click_train)

3485852

In [15]:
len(click_validation)

NameError: name 'click_validation' is not defined

In [16]:
len(data_click)

3485852

In [57]:
click_feat_list = CATEGORIES + BINARY + NUMERICAL#['f_43','f_48','f_50','f_66','f_68','f_69','f_70','f_72','f_73']
install_feat_list = CATEGORIES + BINARY + NUMERICAL#['f_58','f_59','f_50','f_68']

In [20]:
use_feat = []
# use_feat.extend(DATE)
use_feat.extend(CATEGORIES)
use_feat.extend(BINARY)
use_feat.extend(NUMERICAL)

In [20]:
len(use_feat)

78

In [17]:
y_click = train[IS_CLICKED[0]]
y_install = train[IS_INSTALLED[0]]
X = train[use_feat]

NameError: name 'train' is not defined

In [ ]:
X.columns

In [18]:
from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score
# from sklearn.naive_bayes import ComplementNB,MultinomialNB
# from sklearn.linear_model import LogisticRegression

# Naive Bayes

In [ ]:
model_click = MultinomialNB(alpha=1)
model_click.fit(X,y_click)
print("Done Click")
model_install = MultinomialNB(alpha=1)
model_install.fit(X,y_install)
print("Done Install")

## Logistic Regression

In [ ]:

model_click = LogisticRegression(class_weight={1:2,0:1}).fit(train[NUMERICAL],y_click)
model_install = LogisticRegression(class_weight={1:2,0:1}).fit(train[NUMERICAL],y_install)

## XGBoost

In [21]:
model_click = XGBClassifier(learning_rate=0.1, n_estimators=100, objective='binary:logistic',
                            verbosity=2,tree_method='gpu_hist')#,scale_pos_weight=3)
model_click.fit(click_train[use_feat],click_train[IS_CLICKED])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [18]:
y_click_pred = model_click.predict(install_train[click_feat_list])
install_train.loc[:,['is_click_pred']] = y_click_pred

/tmp/ipykernel_23082/688394720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  install_train.loc[:,['is_click_pred']] = y_click_pred
/tmp/ipykernel_23082/688394720.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  install_train.loc[:,['is_click_pred']] = y_click_pred


In [ ]:
train.columns

In [22]:
model_install = XGBClassifier(learning_rate=0.1, n_estimators=100, objective='binary:logistic',
                              verbosity=2,tree_method='gpu_hist')
model_install.fit(install_train[use_feat],install_train[IS_INSTALLED])# +['is_click_pred']

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Random Forest

In [ ]:
rf_install = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf_install.fit(X,y_install)

In [ ]:
model_install = rf_install

In [ ]:
model_click = RandomForestClassifier(n_estimators=100,n_jobs=-1)
model_click.fit(X,y_click)

In [ ]:
X_val = validation[use_feat]

In [ ]:
y_click_val = validation[IS_CLICKED[0]]
y_install_val = validation[IS_INSTALLED[0]]

In [ ]:
y_click_pred = model_click.predict(validation[use_feat])
y_install_pred = model_install.predict(validation[use_feat])

# Naive Baysem

In [ ]:
y_click_pred = nb_click.predict(X_val)
y_install_pred = nb_install.predict(X_val)

## XGB

In [21]:
y_click_pred = model_click.predict(click_validation[click_feat_list])
# validation['is_click_pred'] = y_click_pred
y_install_pred = model_install.predict(install_validation[install_feat_list])

In [22]:
import numpy as np
np.unique(y_click_pred,return_counts=True)

(array([0, 1]), array([65334, 32638]))

In [23]:
np.unique(y_install_pred,return_counts=True)

(array([0, 1]), array([92829,  5143]))

In [24]:
from sklearn.metrics import f1_score
print(f1_score(click_validation[IS_CLICKED],y_click_pred))

0.6222965636646419


In [25]:
print(f1_score(install_validation[IS_INSTALLED],y_install_pred))

0.25601973964094277


# Test

In [23]:
# import pandas as pd
test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')

## Missing Value Imputation for Test

In [24]:
test['f_30'].fillna(install_train['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(install_train['f_31'].mode()[0],inplace=True)

In [25]:
for f in fmiss:
    test[f].fillna(install_train[f].mean(),inplace=True)

In [26]:
test.columns

Index(['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9',
       'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 'f_17', 'f_18',
       'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_27',
       'f_28', 'f_29', 'f_30', 'f_31', 'f_32', 'f_33', 'f_34', 'f_35', 'f_36',
       'f_37', 'f_38', 'f_39', 'f_40', 'f_41', 'f_42', 'f_43', 'f_44', 'f_45',
       'f_46', 'f_47', 'f_48', 'f_49', 'f_50', 'f_51', 'f_52', 'f_53', 'f_54',
       'f_55', 'f_56', 'f_57', 'f_58', 'f_59', 'f_60', 'f_61', 'f_62', 'f_63',
       'f_64', 'f_65', 'f_66', 'f_67', 'f_68', 'f_69', 'f_70', 'f_71', 'f_72',
       'f_73', 'f_74', 'f_75', 'f_76', 'f_77', 'f_78', 'f_79'],
      dtype='object')

In [27]:
k = test.isnull().sum()

In [28]:
for i,a in enumerate(k):
    print(f"f_{i} {a}")

f_0 0
f_1 0
f_2 0
f_3 0
f_4 0
f_5 0
f_6 0
f_7 0
f_8 0
f_9 0
f_10 0
f_11 0
f_12 0
f_13 0
f_14 0
f_15 0
f_16 0
f_17 0
f_18 0
f_19 0
f_20 0
f_21 0
f_22 0
f_23 0
f_24 0
f_25 0
f_26 0
f_27 0
f_28 0
f_29 0
f_30 0
f_31 0
f_32 0
f_33 0
f_34 0
f_35 0
f_36 0
f_37 0
f_38 0
f_39 0
f_40 0
f_41 0
f_42 0
f_43 0
f_44 0
f_45 0
f_46 0
f_47 0
f_48 0
f_49 0
f_50 0
f_51 0
f_52 0
f_53 0
f_54 0
f_55 0
f_56 0
f_57 0
f_58 0
f_59 0
f_60 0
f_61 0
f_62 0
f_63 0
f_64 0
f_65 0
f_66 0
f_67 0
f_68 0
f_69 0
f_70 0
f_71 0
f_72 0
f_73 0
f_74 0
f_75 0
f_76 0
f_77 0
f_78 0
f_79 0


In [30]:
y_click_test = model_click.predict_proba(test[use_feat])

In [24]:
test['is_click_pred'] = model_click.predict(test[click_feat_list])

In [31]:
y_install_test = model_install.predict_proba(test[use_feat])#+['is_click_pred']

In [32]:
# y_click_test = nb_click.predict(test[use_feat])
# y_install_test = nb_install.predict(test[use_feat])

In [32]:
import numpy as np
result = np.vstack([test['f_0'].to_numpy(dtype=int),y_click_test[:,1],y_install_test[:,1]])

In [33]:
result.shape

(3, 160973)

In [34]:
result = result.T

In [35]:
result.shape

(160973, 3)

In [36]:
final = pd.DataFrame(result,columns=['RowId','is_clicked','is_installed'])

In [37]:
final.head()

,RowId,is_clicked,is_installed
0,64505.0,0.281865,0.212022
1,64506.0,0.077323,0.255021
2,64507.0,0.277757,0.101083
3,64508.0,0.131246,0.233965
4,64509.0,0.334293,0.113833


In [38]:
final['RowId']=final['RowId'].astype('int')

In [39]:
final.to_csv('../Data/xgb_100.csv', sep ='\t', index=False)